# Dunham's Data

## Setting up

In [1]:
from geopy.distance import geodesic
# import networkx as nx
import pandas as pd
# import seaborn as sns

In [2]:
# pd.set_option('display.max_rows', 999)

In [3]:
# sns.set()

In [4]:
# %matplotlib inline

## Loading data

In [5]:
col_used = [
    'DATE', 'CITY1', 'COUNTRY1', 'CITY2', 'COUNTRY2', 'WORKING', 'HOTEL_ADDRESS',
    'VENUE1', 'VENUE_TYPE1', 'VENUE2', 'VENUE_TYPE2', 'NOTES', 'SOURCE',
]
dunham_df = pd.read_excel('data/src/DunhamsData_EverydayItinerary_DatasetSample.xlsx',
                          usecols='A:G,I:N', names=col_used)
dunham_df.fillna('', inplace=True)

In [6]:
dunham_df.shape

(10, 13)

In [7]:
dunham_df.head()

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
0,1947-03-14,New York City,USA,,,y,,Hotel Delmonico,n,Roxy,n,"Diary--""All Star Show for the Wounded"" between...",Display Ad New York Times (1923-Current file);...
1,1947-04-07,New York City,USA,,,y,,,,,,Diary--rehearsal,SIU Box 142 Folder 7
2,1947-06-28,Mexico City,Mexico,,,y,"Arenal #7, San Angel, Mexico, D.F.",,,,,,
3,1947-07-01,Mexico City,Mexico,,,y,"Arenal #7, San Angel, Mexico, D.F.",,,,,Rehearsing every day at Dunham's rented house;...,SIU Box 10 Folder 4
4,1947-07-06,Mexico City,Mexico,,,y,"Arenal #7, San Angel, Mexico, D.F.",,,,,,


In [8]:
dunham_df.tail(1)

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
9,1948-01-04,San Francisco,USA,,,y,,Geary Theater,c,,,,Bal Negre program


## Pre-processing data

In [9]:
dunham_df.DATE = dunham_df.DATE.astype(str)
for col in dunham_df.columns[1:]:
    dunham_df[col] = dunham_df[col].apply(str.strip)

In [10]:
def clean_city(x, y):
    if x == 'Hamilton':
        if y == 'Canada':
            r = 'Hamilton (CA)'
        else: # y == 'New Zealand'
            r = 'Hamilton (NZ)'
#     elif x == 'in transit':
#         r = ''
    else:
        r = x
    return r

dunham_df.CITY1 = dunham_df.apply(lambda row: clean_city(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.CITY2 = dunham_df.apply(lambda row: clean_city(row.CITY2, row.COUNTRY2), axis=1)

In [11]:
def clean_country(city, country):
    x = country
    y = city
    if y == 'Gibraltar':
        r = 'Gibraltar'
    elif x == 'Newfoundland':
        r = 'Canada'
    else:
        r = x
    return r

dunham_df.COUNTRY1 = dunham_df.apply(lambda row: clean_country(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.COUNTRY2 = dunham_df.apply(lambda row: clean_country(row.CITY2, row.COUNTRY2), axis=1)

In [12]:
# def clean_working(x):
#     if x.startswith('[[') and x.endswith(']]'):
#         r = ''
#     else:
#         r = x
#     return r

# dunham_df.WORKING = dunham_df.WORKING.apply(clean_working)

In [13]:
def clean_hotel_address(x):
    r = x # do not clean for now
    return r

dunham_df.HOTEL_ADDRESS = dunham_df.HOTEL_ADDRESS.apply(clean_hotel_address)

In [14]:
# def clean_performance_yn(x):
#     if x == '?':
#         r = ''
#     elif x == 'N':
#         r = 'n'
#     else:
#         r = x
#     return r

# dunham_df.PERFORMANCE_YN = dunham_df.PERFORMANCE_YN.apply(clean_performance_yn)

In [15]:
# def clean_theater1(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER1 = dunham_df.THEATER1.apply(clean_theater1)

In [16]:
# def clean_theater2(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER2 = dunham_df.THEATER2.apply(clean_theater2)

In [17]:
# def clean_mode_transit(x):
#     if x == '[[chartered bus Reno to Los Angeles?]]':
#         r = 'bus'
#     elif x == 'drive':
#         r = 'car'
#     elif x == 'drive?':
#         r = 'car'
#     elif x.startswith('plane'):
#         r = 'plane'
#     elif x == 'train; boat train':
#         r = 'train, boat, train'
#     elif x == 'train; plane; boat':
#         r = 'train, plane, boat'
#     else:
#         r = x
#     return r

# dunham_df.MODE_TRANSIT = dunham_df.MODE_TRANSIT.apply(clean_mode_transit)

In [18]:
# def clean_transit_cities(x):
#     if x == 'Geneva; Rome':
#         r = 'Geneva (Switzerland), Rome (Italy)'
#     elif x == 'transit to Turin':
#         r = ''
#     elif x == 'transit through London':
#         r = 'London (England)'
#     elif x == 'transit: PanAm 201 flies NYC Laguardia - San Juan - Port of Spain - Belem':
#         r = 'San Juan (Puerto Rico), Port of Spain (Trinidad and Tobago), Belem (Brazil)'
#     elif x == 'Los Cerrillos airport':
#         r = ''
#     elif x == 'transit':
#         r = ''
#     elif x == 'destination':
#         r = ''
#     elif x == 'transit via Boston':
#         r = 'Boston (USA)'
#     elif x == 'in transit':
#         r = ''
#     elif x == 'last performance':
#         r = ''
#     elif x == 'London (Liverpool station), England; Harwich (Harwich Quay), England':
#         r = 'Harwich (England)'
#     elif x == 'New York':
#         r = ''
#     elif x == 'Chicago':
#         r = 'Chicago (USA)'
#     elif x == 'Gander, Newfoundland, Canada':
#         r = 'Gander (Canada)'
#     elif x == 'Zurich, Switzerland':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Zurich':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Rome, Italy; Geneva, Switzerland':
#         r = 'Rome (Italy), Geneva (Switzerland)'
#     elif x == 'Vevey, Switzerland':
#         r = 'Vevey (Switzerland)'
#     elif x == 'Amsterdam, Netherlands':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Furet de P*?': # ???
#         r = ''
#     elif x == 'San Diego; Tijuana':
#         r = 'San Diego (USA), Tijuana (Mexico)'
#     elif x == 'Portworth':
#         r = 'Fort Worth (USA)'
#     elif x == 'Miami':
#         r = 'Miami (USA)'
#     elif x == 'Los Angeles':
#         r = 'Los Angeles (USA)'
#     elif x == "El Paso, TX----I think probably this was to go to Mexico, and I don't think she ever went to Mexico":
#         r = 'El Paso (USA)'
#     elif x == 'Honolulu':
#         r = 'Honolulu (Hawaii)'
#     elif x == 'Coolangatta':
#         r = 'Coolangatta (Australia)'
#     elif x == '[[Dunham Brisbane to Perth via Sydney and Melbourne--date open: https://osu.app.box.com/file/325139873147]]':
#         r = ''
#     elif x == 'Malacca':
#         r = 'Malacca (Malaysia)'
#     elif x == 'Amsterdam':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Lyon; Bordeaux; Bayonne':
#         r = 'Lyons (France), Bordeaux (France), Bayonne (France)'
#     elif x == 'Irun, Spain':
#         r = 'Irun (Spain)'
#     elif x == 'Irun, Spain; Hendaye, France':
#         r = 'Irun (Spain), Hendaye (France)'
#     elif x == 'Marseilles':
#         r = 'Marseille (France)'
#     elif x == 'Piraeus, Greece':
#         r = 'Piraeus (Greece)'
#     elif x == 'Port Said, Egypt':
#         r = 'Port Said (Egypt)'
#     elif x == 'Munich':
#         r = 'Munich (Germany)'
#     elif x == 'Schwarzbach; Ravensberg':
#         r = 'Schwarzbach (Germany), Ravensberg (Germany)'
#     elif x == 'Colmar; Ligny en Barois; Rosay en Brie':
#         r = 'Colmar (France), Ligny-en-Barrois (France), Rozay-en-Brie (France)'
#     elif x == 'Verneson; Valence':
#         r = 'Vernaison (France), Valence (France)'
#     elif x == 'Alicante':
#         r = 'Alicante (Spain)'
#     elif x == 'Granada':
#         r = 'Granada (Spain)'
#     elif x == 'La Linea, Spain':
#         r = 'La Linea (Spain)'
#     else:
#         r = x
#     return r

# dunham_df.TRANSIT_CITIES = dunham_df.TRANSIT_CITIES.apply(clean_transit_cities)

In [19]:
def clean_notes(x):
    r = x # do not clean for now
    return r

dunham_df.NOTES = dunham_df.NOTES.apply(clean_notes)

In [20]:
def clean_source(x):
    r = x # do not clean for now
    return r

dunham_df.SOURCE = dunham_df.SOURCE.apply(clean_source)

In [21]:
dunham_df.fillna('', inplace=True)

## Processing data

In [22]:
geolocations = { # (lat, lon)
    'Acapulco': (16.863611, -99.8825),
    'Adelaide': (-34.928889, 138.601111),
    'Algiers': (36.753889, 3.058889),
    'Alicante': (38.345278, -0.483056),
    'Altreu': (47.191818, 7.449104),
    'Annecy': (45.916, 6.133),
    'Amsterdam': (52.366667, 4.9),
    'Antwerp': (51.217778, 4.400278),
    'Arcachon': (44.65, -1.17),
    'Arles': (43.6767, 4.6278),
    'Athens': (37.983972, 23.727806),
    'Auckland': (-36.840556, 174.74),
    'Barcelona': (41.383333, 2.183333),
    'Bari': (41.125278, 16.866667),
    'Barranquilla': (10.963889, -74.796389),
    'Basel': (47.566667, 7.6),
    'Bayonne': (43.49, -1.48),
    'Beirut': (33.886944, 35.513056),
    'Belem': (-1.455833, -48.503889),
    'Belfast': (54.598611, -5.9275),
    'Belgrade': (44.816667, 20.466667),
    'Bergamo': (45.695, 9.67),
    'Bern': (46.95, 7.45),
    'Beverly Hills': (34.073056, -118.399444),
    'Biarritz': (43.48, -1.56),
    'Biel/Bienne': (47.133333, 7.25),
    'Biella': (45.566667, 8.066667),
    'Birmingham': (52.483056, -1.893611),
    'Bogota': (4.711111, -74.072222),
    'Bologna': (44.5075, 11.351389),
    'Bordeaux': (44.84, -0.58),
    'Boston': (42.358056, -71.063611),
    'Bournemouth': (50.72, -1.88),
    'Brescia': (45.541667, 10.216667),
    'Brisbane': (-27.467778, 153.028056),
    'Bristol': (51.45, -2.583333),
    'Brussels': (50.85, 4.35),
    'Buenos Aires': (-34.603333, -58.381667),
    'Buffalo': (42.904722, -78.849444),
    'Cairnryan': (54.971, -5.018),
    'Cairo': (30.044444, 31.235833),
    'Cali': (3.420556, -76.522222),
    'Cannes': (43.5513, 7.0128),
    'Capri': (40.55, 14.233333),
    'Caracas': (10.480556, -66.903611),
    'Carbondale, IL': (37.726, -89.22),
    'Cartagena': (10.4, -75.5),
    'Casablanca': (33.533333, -7.583333),
    'Catania': (37.5, 15.090278),
    'Chicago': (41.9483, -87.6574),
    'Christchurch': (-43.53, 172.620278),
    'Cleveland': (41.482222, -81.669722),
    'Coolangatta': (-28.166667, 153.533333),
    'Colmar': (48.0817, 7.3556),
    'Columbus, OH': (39.962222, -83.000556),
    'Copenhagen': (55.676111, 12.568333),
    'Crystal Bay, NV': (39.227778, -120.004444),
    'Dax': (43.71, -1.05),
    'Denver': (39.76185, -104.881105),
    'Detroit': (42.331389, -83.045833),
    'Dongducheon': (37.916667, 127.066667),
    'Dublin': (53.349722, -6.260278),
    'Dusseldorf': (51.233333, 6.783333),
    'Edinburgh': (55.953, -3.189),
    'El Paso': (31.759167, -106.488611),
    'Esmeraldas': (0.95, -79.666667),
    'Florence': (43.783333, 11.25),
    'Fiesole': (43.807222, 11.291944),
    'Foggia': (41.464167, 15.546111),
    'Fort Worth': (32.75, -97.333333),
    'Frankfurt': (50.116667, 8.683333),
    'Frauenfeld': (47.55, 8.9),
    'Gander': (48.956944, -54.608889),
    'Geneva': (46.2, 6.15),
    'Genoa': (44.411111, 8.932778),
    'Gibraltar': (36.131667, -5.351667),
    'Glasgow': (55.860916, -4.251433),
    'Granada': (37.178056, -3.600833),
    'Guadalajara': (20.666667, -103.35),
    'Guanajuato': (21.017778, -101.256667),
    'Hamburg': (53.565278, 10.001389),
    'Hamilton (CA)': (43.256667, -79.869167),
    'Hamilton (NZ)': (-37.783333, 175.283333),
    'Hanover': (52.366667, 9.716667),
    'Harwich': (51.94458, 1.289852),
    'Havana': (23.133333, -82.383333),
    'Heidelberg': (49.416667, 8.716667),
    'Hendaye': (43.36, -1.77),
    'Hollywood': (34.1, -118.333333),
    'Hong Kong': (22.3, 114.2),
    'Honolulu': (21.3, -157.816667),
    'Indianapolis': (39.768611, -86.158056),
    'Interlaken': (46.683333, 7.85),
    'Irun': (43.337814, -1.788811),
    'Ito': (34.965722, 139.101861),
    'Joliet': (41.520556, -88.150556),
    'Juan-les-Pins': (43.571111, 7.109444),
    'Kiamesha Lake, NY': (41.683056, -74.660833),
    'Kingston': (18, -76.783333),
    'Knokke': (51.341389, 3.286944),
    'Kuala Lumpur': (3.147778, 101.695278),
    'La Baule-Escoublac': (47.2867, -2.3908),
    'La Linea': (36.168056, -5.348611),
    'La Tour-de-Peilz': (46.45, 6.866667),
    'Lagos': (6.455027, 3.384082),
    'Las Vegas': (36.175, -115.136389),
    'Lausanne': (46.519833, 6.6335),
    'Lecce': (40.35, 18.166667),
    'Les Baux de Provence': (43.7439, 4.7953),
    'Liege': (50.633333, 5.566667),
    'Ligny-en-Barrois': (48.6897, 5.325),
    'Lima': (-12.043333, -77.028333),
    'Liverpool': (53.4, -2.983333),
    'Livorno': (43.55, 10.316667),
    'London': (51.507222, -0.1275),
    'Los Angeles': (34.05, -118.25),
    'Los Angeles area': (34.05, -118.25),
    'Lugano': (46, 8.95),
    'Lyons': (45.76, 4.84),
    'Madrid': (40.383333, -3.716667),
    'Malacca': (2.2, 102.25),
    'Malaga': (36.719444, -4.42),
    'Manchester': (53.479444, -2.245278),
    'Manila': (14.5958, 120.9772),
    'Mannheim': (49.488889, 8.469167),
    'Mar del Plata': (-38.0, -57.55),
    'Marseille': (43.2964, 5.37),
    'Medellin': (6.230833, -75.590556),
    'Melbourne': (-37.813611, 144.963056),
    'Mexico City': (19.433333, -99.133333),
    'Miami': (25.775278, -80.208889),
    'Miami Beach': (25.813025, -80.134064),
    'Milan': (45.466667, 9.183333),
    'Monte Carlo': (43.739722, 7.427222),
    'Monte Porzio': (43.683333, 13.05),
    'Montevideo': (-34.883611, -56.181944),
    'Montreux': (46.433333, 6.916667),
    'Mosman': (-33.829075, 151.24409),
    'Munich': (48.133333, 11.566667),
    'Naples': (40.845, 14.258333),
    'New York City': (40.7127, -74.0059),
    'Newark': (40.72, -74.17),
    'Nassau': (25.06, -77.345),
    'Nicosia': (35.1725, 33.365),
    'Nice': (43.7034, 7.2663),
    'Nimes': (43.838, 4.361),
    'Nottingham': (52.953333, -1.15),
    'Nuremberg': (49.45, 11.083333),
    'Oakland, CA': (37.804444, -122.270833),
    'Okinawa': (26.479444, 127.927778),
    'Oran': (35.696944, -0.633056),
    'Padova': (45.416667, 11.866667),
    'Palermo': (38.116667, 13.366667),
    'Palm Springs': (33.830278, -116.545278),
    'Palma, Majorca': (39.566667, 2.65),
    'Paris': (48.8567, 2.3508),
    'Perth': (-31.952222, 115.858889),
    'Petion-Ville': (18.516667, -72.283333),
    'Philadelphia': (39.952778, -75.163611),
    'Piacenza': (45.047778, 9.700556),
    'Piraeus': (37.943, 23.646944),
    'Pittsburgh': (40.439722, -79.976389),
    'Pontillac': (44.1383, 4.8097),
    'Portland, OR': (45.52, -122.681944),
    'Port-au-Prince': (18.533333, -72.333333),
    'Port of Spain': (10.666667, -61.516667),
    'Port Said': (31.25, 32.3),
    'Quito': (-0.233333, -78.516667),
    'Ravensberg': (52.092164, 8.570708),
    'Recife': (-8.05, -34.9),
    'Reno': (39.527222, -119.821944),
    'Rio de Janeiro': (-22.908333, -43.196389),
    'Rome': (41.9, 12.5),
    'Rozay-en-Brie': (48.6833, 2.9667),
    'San Antonio, Ibiza': (38.980833, 1.300556),
    'San Diego': (32.715, -117.1625),
    'San Francisco': (37.783333, -122.416667),
    'San Jose': (37.333333, -121.9),
    'San Juan': (18.406389, -66.063889),
    'Santa Margherita': (44.333333, 9.216667),
    'Santander': (43.462778, -3.805),
    'Santiago': (-33.45, -70.666667),
    'Santos': (-23.936944, -46.325),
    'São Paulo': (-23.55, -46.633333),
    'Sarajevo': (43.866667, 18.416667),
    'Sayama': (35.852972, 139.412194),
    'Schaffhausen': (47.7, 8.633333),
    'Schwarzbach': (51.45, 13.933056),
    'Seattle': (47.609722, -122.333056),
    'Seoul': (37.566667, 126.966667),
    'Sicily': (37.5, 14.0),
    'Sidi Bel Abbès': (35.193889, -0.641389),
    'Singapore': (1.283333, 103.833333),
    'Somerville, NJ': (40.56975, -74.607682),
    "St. Catherine's": (43.183333, -79.233333),
    'St. Louis': (38.627222, -90.197778),
    'Stockholm': (59.329444, 18.068611),
    'Stockton, CA': (37.975556, -121.300833),
    'Stuttgart': (48.783333, 9.183333),
    'Surrey': (51.25, -0.416667),
    'Sydney': (-33.865, 151.209444),
    'Tachikawa': (35.694222, 139.419667),
    'Tacoma, WA': (47.241389, -122.459444),
    'Tangier': (35.766667, -5.8),
    'The Hague': (52.083333, 4.316667),
    'Tijuana': (32.525, -117.033333),
    'Tivoli': (41.959722, 12.798056),
    'Tokyo': (35.689722, 139.692222),
    'Toronto': (43.741667, -79.373333),
    'Trieste': (45.633333, 13.8),
    'Turin': (45.079167, 7.676111),
    'Utrecht': (52.083333, 5.116667),
    'Valence': (44.9333, 4.8917),
    'Valencia': (39.466667, -0.375),
    'Vancouver': (49.25, -123.1),
    'Varese': (45.816667, 8.833333),
    'Venice': (45.4375, 12.335833),
    'Ventimiglia': (43.790278, 7.608333),
    'Veracruz': (19.433333, -96.383333),
    'Vernaison': (45.648, 4.811),
    'Verona': (45.433333, 10.983333),
    'Vevey': (46.466667, 6.85),
    'Vichy': (46.1278, 3.4267),
    'Vienna': (48.2, 16.366667),
    'Washington, D.C.': (38.904722, -77.016389),
    'Wellington': (-41.288889, 174.777222),
    'West Berlin': (52.5333, 13.1667),
    'Yokohama': (35.444167, 139.638056),
    'Zurich': (47.366667, 8.55),
}

### Grouping locations

In [23]:
distances = set()
cities = set(dunham_df.CITY1.unique()).union(dunham_df.CITY2.unique())
cities.discard('')
cities.discard('in transit')
for i, city1 in enumerate(cities):
    p1 = geolocations[city1]
    for j, city2 in enumerate(cities):
        if j > i: # and city1 != city2:
            p2 = geolocations[city2]
            dst = geodesic(p1, p2).km
            distances.add((city1, city2, dst))

In [24]:
len(distances)

10

In [25]:
sorted(distances, key=lambda x: x[2])[:10]

[('Stockton, CA', 'San Francisco', 100.45777277568092),
 ('San Diego', 'Stockton, CA', 694.1804200964181),
 ('San Diego', 'San Francisco', 737.8514518146319),
 ('San Diego', 'Mexico City', 2321.4317013516998),
 ('Stockton, CA', 'Mexico City', 2970.451620820282),
 ('Mexico City', 'San Francisco', 3036.6053907108394),
 ('New York City', 'Mexico City', 3357.6419513494357),
 ('New York City', 'San Diego', 3915.5011469250653),
 ('New York City', 'Stockton, CA', 4039.0194401772533),
 ('New York City', 'San Francisco', 4138.60863656365)]

In [26]:
THRESHOLD = 15

In [27]:
grouped_cities_dct = {
    'Petion-Ville': 'Port-au-Prince',
    'Los Angeles': 'Los Angeles area',
    'Hollywood': 'Los Angeles area',
    'Beverly Hills': 'Los Angeles area',
    'Fiesole': 'Florence',
#     'Mosman': 'Sydney',
}

In [28]:
for k in grouped_cities_dct:
    dunham_df.replace(k, grouped_cities_dct[k], inplace=True)

In [29]:
dunham_df.shape

(10, 13)

## Saving data

In [30]:
dunham_df.to_csv('data/out/dunham_1947-60.csv', index=False)